In [1]:
from pyClarion import (Agent, Input, Choice, ChunkStore, FixedRules, 
    Family, Atoms, Atom, BaseLevel, Pool, NumDict, Event, Priority, Site, IDN, Train)
from pyClarion.components.stats import MatchStats
from datetime import timedelta

import logging
import sys

import numpy as np
import pandas as pd
from typing import *

from utils import RuleWBLA, numpify_grid
from knowledge_init import *
from rule_defs import * 
import math

In [2]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(funcName)s - %(message)s')

logger = logging.getLogger("pyClarion.system")
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
# handler.setFormatter(formatter)
logger.addHandler(handler)

In [3]:
class BrickConstructionTask(Family):
    bricks: Brick
    brick_nos: NumBricksInput
    grid_rows: GridRows
    grid_cols: GridCols
    signal_tokens: SignalTokens
    io: ConstructionIO

class BrickResponseTask(Family):
    bricks: Brick
    grid_rows: GridRows
    grid_cols: GridCols
    query_rel: QueryRel
    io: ResponseIO
    response: Response


In [21]:
class Participant(Agent):  
    construction_space: BrickConstructionTask
    response_space: BrickResponseTask
    construction_input: Input
    response_input: Input
    response_rules: RuleWBLA
    response_blas: BaseLevel
    pool: Pool
    response_choice: Choice
    #TODO: fill in the rest here once ur done below

    def __init__(self, name: str) -> None:
        p = Family() # what is p
        e = Family() # what is e?
        r_construction  = Family() # rule family for construction rules
        r_response = Family() # rule family for response rules
        c_construction = Family() # construction family for construction chunks
        c_response = Family() # response family for response chunks
        construction_space = BrickConstructionTask()
        response_space = BrickResponseTask()

        #RL components
        h = Family() # hidden layer information

        super().__init__(name, p=p, e=e, h=h,
                         construction_space=construction_space, response_space=response_space, 
                         r_construction=r_construction, r_response=r_response, 
                         c_construction=c_construction, c_response=c_response)
        self.construction_space = construction_space
        self.response_space = response_space
        with self:
            self.construction_input = Input("construction_input", (construction_space, construction_space))
            self.response_input = Input("response_input",  (response_space, response_space), reset=False)
            self.response_rules = RuleWBLA("response_rules", p=p, r=r_response, c=c_response, d=response_space, v=response_space, sd=1e-4)
            self.search_space_rules = RuleWBLA("search_space_rules", p=p, r=r_construction, c=c_construction, d=construction_space, v=construction_space, sd=1e-4)
            
            self.response_blas = BaseLevel("blas", p, e, self.response_rules.rules.rules)
            self.search_space_blas = BaseLevel("search_space_blas", p, e, self.search_space_rules.rules.rules)
            self.response_blas.ignore.add(~self.response_rules.rules.rules.nil) 
            self.search_space_blas.ignore.add(~self.search_space_rules.rules.rules.nil)

            #TODO: this good?
            self.search_space_matchstats = MatchStats("search_space_matchstats", p, self.search_space_rules.rules.rules, th_cond=0.9, th_crit=0.9) # 0.9 because i want to compare against 1.
            
            self.response_pool = Pool("pool", p, self.response_rules.rules.rules, func=NumDict.sum) # to pool together the blas and condition activations
            self.search_space_pool = Pool("search_space_pool", p, self.search_space_rules.rules.rules, func=NumDict.sum) # similar function

            self.response_choice = Choice("choice", p, (response_space, response_space))
            self.search_space_choice = Choice("search_space_choice", p, (construction_space, construction_space))

            # RL Construction
            with self.search_space_rules.choice:
                self.construction_net = self.construction_input >> IDN("construction_net", 
                                                                    p=p, 
                                                                    h=h,
                                                                    r=self.search_space_rules.rules.rules, # reward specification on rule choice
                                                                    s1=(self.construction_space, self.construction_space), #input construction space has goal state and current state
                                                                    s2=self.search_space_rules.rules.rules,
                                                                    layers=(1024, 512, 256, 512, 1024),
                                                                    train=Train.WEIGHTS,
                                                                    gamma=.3,
                                                                    lr=1e-2
                                                                    )
        
        self.response_blas.input = self.response_rules.choice.main
        self.search_space_blas.input = self.search_space_rules.choice.main

        self.response_rules.rules.lhs.bu.input = self.response_input.main 
        self.search_space_rules.rules.lhs.bu.input = self.construction_input.main

        self.search_space_pool["search_spaces_rules.rules"] = (
            self.search_space_rules.rules.main,  
            lambda d: d.shift(x=1).scale(x=0.5).logit())
        self.response_pool["response_rules.rules"] = (
            self.response_rules.rules.main,
            lambda d: d.shift(x=1).scale(x=0.5).logit())
        
        self.response_pool["blas"] = (
            self.response_blas.main, 
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0))
        self.search_space_pool["blas"] = (
            self.search_space_blas.main,
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0))
        
        self.search_space_pool["search_space_matchstats"] = (
            self.search_space_matchstats.main, 
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0)) # TODO: is the function here correct?
        self.search_space_pool["search_space_netscores"] = (
            self.construction_net.olayer.main,
            lambda d: d.bound_min(x=1e-8).log().with_default(c=0.0)) # TODO: verify this scaling..
        
        self.response_rules.bla_main = self.response_pool.main #updated site for the response rules to take BLAS into account
        self.search_space_rules.bla_main = self.search_space_pool.main
        #rewire choice input
        self.search_space_rules.choice.input = self.search_space_pool.main
        self.response_rules.choice.input = self.response_pool.main

        self.response_choice.input = self.response_rules.rules.rhs.td.main # bu choice
        self.search_space_choice.input = self.search_space_rules.rules.rhs.td.main

        with self.response_pool.params[0].mutable():
            self.response_pool.params[0][~self.response_pool.p["blas"]] = 2e-1
        with self.search_space_pool.params[0].mutable():
            self.search_space_pool.params[0][~self.search_space_pool.p["blas"]] = 2e-1
        
        #backtracking queues: #TODO: better way to do this is probably with Sites, adn their inbuilt deque
        self.past_constructions = []
        self.past_chosen_rules = []
        self.all_rule_history = []

        #RL stats
        self.construction_reward_vals = []
        self.construction_qvals = []
        self.construction_net_training_results = []

    def resolve(self, event: Event) -> None:
        # -- RESPONSE PROCESSING --
        if event.source == self.response_rules.rules.update: # after the rules have updated 
            self.response_blas.update() # timestep update
        elif event.source == self.response_pool.update: # TODO: is this right?
            self.response_rules.trigger()
        elif event.source == self.response_rules.rules.rhs.td.update:
            self.response_choice.trigger() #poll outside the loop in experimental loop
        
        # -- SEARCH PROCESSING --
        elif event.source == self.search_space_rules.rules.update:
            if not self.past_constructions: self.past_constructions = [self.construction_input.main[0]]
            self.search_space_blas.update()
            self.search_space_matchstats.update() # update the match stats
        
        elif event.source == self.search_space_pool.update:
            self.search_space_rules.trigger()
        
        elif event.source == self.search_space_rules.rules.rhs.td.update:
            self.search_space_choice.trigger()
        
        elif event.source == self.search_space_choice.select:
            #check if indeed we need to stop construction, if triggered the STOP rule
            cur_sample = self.search_space_rules.choice.sample
            cur_rule_choice = self.search_space_rules.choice.poll()
            
            temp_sample = cur_sample.new(cur_sample[0]._d).with_default(c=-math.inf)
            temp_sample = temp_sample.exp().div(temp_sample.exp().sum())
            cur_sample.data.pop()
            cur_sample.data.append(temp_sample) # simple copy for softmaxing activation

            cur_choice = self.search_space_choice.poll()

            if cur_sample[0][list(cur_rule_choice.values())[0]] < 0.2: #backtrack # TODO: guaranteed to be list of size 1 yea?
                new_rule_mask = self.search_space_matchstats.cond.new({list(cur_rule_choice.values())[0]: 1.0}).with_default(c=0.0)
                new_crit_score = self.search_space_matchstats.crit.new({}).with_default(c=0.0)#we want to increment the negativity count
                
                self.search_space_matchstats.cond.data.pop()
                self.search_space_matchstats.crit.data.pop()

                self.search_space_matchstats.cond.data.append(new_rule_mask) # update the condition ot only change scores for this rule
                self.search_space_matchstats.crit.data.append(new_crit_score) 

                self.search_space_matchstats.increment() # TODO: apt timedelta?
                self.construction_input.send(self.past_constructions.pop()) # pop the last construction

                # also give -1 reward to the rule that was chosen 
                self.construction_net.error.send({list(cur_rule_choice.values())[0]: -1.0}) # TODO: check this
                self.construction_reward_vals.append(-1.0)
                self.construction_qvals.append(self.search_space_pool["search_space_netscores"][0].max().c)

            elif cur_choice[~self.construction_space.io.construction_signal * ~self.construction_space.signal_tokens] == ~self.construction_space.io.construction_signal * ~self.construction_space.signal_tokens.stop_construction: # TODO: check this
                self.end_construction()
                self.response_input.send(self.search_space_choice.main[0]) # send the choice to the response input -- to make a decision out of 
            else: #continue construction
                self.past_constructions.append(self.search_space_choice.main[0])
                self.past_chosen_rules.append(cur_sample[0][list(cur_rule_choice.values())[0]])
                self.all_rule_history.append(cur_sample[0][list(cur_rule_choice.values())[0]])
                self.construction_input.send(self.search_space_choice.main[0]) # loop it back in --for more selections
        
        elif event.source == self.search_space_matchstats.increment:
            # make the mask zero:
            new_empty_mask = self.search_space_matchstats.cond.new({}).with_default(c=0.0)
            self.search_space_matchstats.cond.data.pop()
            self.search_space_matchstats.cond.data.append(new_empty_mask)
            self.search_space_matchstats.discount()#TODO: apt timedelta?

        elif event.source == self.end_construction:
            # clear system queue
            self.system.queue.clear() # at this point, you should have a clear target_grid representation built -- correct or incorrect
        
        elif event.source == self.propagate_feedback:
            self.end_construction_feedback()

        elif event.source == self.construction_net.error.update:
            self.construction_net_training_results.append(self.construction_net.error.main[0].pow(x=2.0))

    def start_construct_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        # the previous construction trial has ended, so clear TD trackers
        self.construction_net.error.reward.data.clear() # TODO: anyway to schedule such changes than doing it here?
        self.construction_net.qvals.data.clear()
        self.construction_net.actions.data.clear()

        self.system.schedule(self.start_construct_trial, dt=dt, priority=priority)

    def end_construction(self, 
                         dt: timedelta = timedelta(seconds=0),
                         priority: Priority = Priority.PROPAGATION
                         ) -> None:
        self.system.schedule(self.end_construction, dt=dt, priority=priority)

    def end_construction_feedback(self,
                                  dt: timedelta = timedelta(seconds=0),
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.end_construction_feedback, dt=dt, priority=priority)

    def start_response_trial(self, 
        dt: timedelta, 
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.start_response_trial, dt=dt, priority=priority)
    
    def finish_response_trial(self,
                              dt: timedelta,
        priority: Priority = Priority.PROPAGATION
    ) -> None:
        self.system.schedule(self.finish_response_trial, dt=dt, priority=priority)

    def propagate_feedback(self, 
                           dt: timedelta = timedelta(seconds=0),
        priority: Priority = Priority.PROPAGATION,
                           correct:float = 0) -> None:
        
        for rule in self.past_chosen_rules:
            new_rule_mask = self.search_space_matchstats.cond[0].new({rule: 1.0}).with_default(c=0.0)
            new_crit_score = self.search_space_matchstats.crit[0].new({}).with_default(c=correct)

            self.search_space_matchstats.cond.data.pop()
            self.search_space_matchstats.crit.data.pop()

            self.search_space_matchstats.cond.data.append(new_rule_mask) # update the condition to only change scores for this rule
            self.search_space_matchstats.crit.data.append(new_crit_score)
            
            self.search_space_matchstats.increment() # TODO: apt timedelta?
            self.search_space_matchstats.discount()
            self.schedule(self.propagate_feedback, dt=dt, priority=priority)

            #update similarly the construction net
            self.construction_net.error.send({rule: correct})
            self.construction_reward_vals.append(correct)
            self.construction_qvals.append(self.search_space_pool["search_space_netscores"][0].max().c)

    #TODO: RL for the construction task + rule extraction from results of RL on the bottom level. 

In [5]:
def present_stimulus(d:BrickConstructionTask, stim_grid: np.ndarray):
    stim_bricks = np.unique(stim_grid)
    stim_bricks = stim_bricks[stim_bricks != 0]

    brick_map = {1: d.bricks.half_T, 2: d.bricks.mirror_L, 3: d.bricks.vertical, 4: d.bricks.horizontal}
    row_map = {1: d.grid_rows.r1, 2: d.grid_rows.r2, 3: d.grid_rows.r3, 4: d.grid_rows.r4, 5: d.grid_rows.r5, 6: d.grid_rows.r6}
    col_map = {1: d.grid_cols.c1, 2: d.grid_cols.c2, 3: d.grid_cols.c3, 4: d.grid_cols.c4, 5: d.grid_cols.c5, 6: d.grid_cols.c6}
    
    brick_row_map = {1: {1: d.io.input_shape1_row1, 2: d.io.input_shape1_row2, 3: d.io.input_shape1_row3}, 2: {1: d.io.input_shape2_row1, 2: d.io.input_shape2_row2, 3: d.io.input_shape2_row3}, 3: {1: d.io.input_shape3_row1, 2: d.io.input_shape3_row2, 3: d.io.input_shape3_row3}, 4: {1: d.io.input_shape4_row1, 2: d.io.input_shape4_row2, 3: d.io.input_shape4_row3}}
    brick_col_map = {1: {1: d.io.input_shape1_col1, 2: d.io.input_shape1_col2, 3: d.io.input_shape1_col3}, 2: {1: d.io.input_shape2_col1, 2: d.io.input_shape2_col2, 3: d.io.input_shape2_col3}, 3: {1: d.io.input_shape3_col1, 2: d.io.input_shape3_col2, 3: d.io.input_shape3_col3}, 4: {1: d.io.input_shape4_col1, 2: d.io.input_shape4_col2, 3: d.io.input_shape4_col3}}
    shape_brick_map = {1: d.io.input_shape1, 2: d.io.input_shape2, 3: d.io.input_shape3, 4: d.io.input_shape4}
    
    in_send_val = None
    for i, brick in enumerate(stim_bricks):
        # brick row indices: 
        row_indices = np.where(stim_grid == brick)[0] + 1
        col_indices = np.where(stim_grid == brick)[1] + 1

        if not in_send_val:
            in_send_val = (+ shape_brick_map[brick] ** brick_map[brick] 
                           + brick_row_map[brick][1] ** row_map[row_indices[0]] 
                           + brick_row_map[brick][2] **  row_map[row_indices[1]]
                           + brick_row_map[brick][3] ** row_map[row_indices[2]]
                           + brick_col_map[brick][1] ** col_map[col_indices[0]]
                           + brick_col_map[brick][2] ** col_map[col_indices[1]]
                           + brick_col_map[brick][3] ** col_map[col_indices[2]])
        else:
            in_send_val = (in_send_val
                            + shape_brick_map[brick] ** brick_map[brick]
                            + brick_row_map[brick][1] ** row_map[row_indices[0]]
                            + brick_row_map[brick][2] ** row_map[row_indices[1]]
                            + brick_row_map[brick][3] ** row_map[row_indices[2]]
                            + brick_col_map[brick][1] ** col_map[col_indices[0]]
                            + brick_col_map[brick][2] ** col_map[col_indices[1]]
                            + brick_col_map[brick][3] ** col_map[col_indices[2]])
            
    return in_send_val

def load_trial(construction_space: BrickConstructionTask, response_space: BrickResponseTask ,trial, t_type="test", q_type="query"):
    d = response_space
    
    grid_name = trial["Grid_Name"]
    
    stim_grid = np.load(f"/Users/mishaal/personalproj/clarion_replay/processed/{t_type}_data/{t_type}_stims/{grid_name}.npy")         
    chunk_grid = present_stimulus(construction_space, stim_grid)
    
    query_map = {1: d.query_rel.left, 2: d.query_rel.above, 3: d.query_rel.right, 4: d.query_rel.below}
    brick_map = {1: d.bricks.half_T, 2: d.bricks.mirror_L, 3: d.bricks.vertical, 4: d.bricks.horizontal}

    if t_type == "test":
        chunk_test = ( + d.io.query_relation ** query_map[trial["Q_Relation"]] 
                      + d.io.query_block ** brick_map[trial["Q_Brick_Left"]]
                      + d.io.query_block_reference ** brick_map[trial["Q_Brick_Right"]])
    elif t_type == "train" and q_type == "query":
        # choose 2 blocks randomly
        blocks = np.random.choice((t := np.unique(stim_grid))[t != 0], 2, replace=False)
        # choose a relation randomly
        relation = np.random.choice([1, 2, 3, 4], 1)
        chunk_test = ( + d.io.query_relation ** query_map[relation[0]] 
                      + d.io.query_block ** brick_map[blocks[0]]
                      + d.io.query_block_reference ** brick_map[blocks[1]])
    else: chunk_test = ()

    print("Stimulus grid: ", stim_grid)
    if q_type == "query" and t_type == "test":
        print("Query brick 1: ", trial["Q_Brick_Left"])
        print("Query brick 2: ", trial["Q_Brick_Right"])
        print("Query relation: ", trial["Q_Relation"])
    elif q_type == "query":
        print("Query brick 1: ", blocks[0])
        print("Query brick 2: ", blocks[1])
        print("Query relation: ", relation[0])

    return stim_grid, chunk_grid, chunk_test

# Simulation

In [6]:
def run_participant_session(participant: Participant, session_df: pd.DataFrame, session_type="train", q_type="query"):
    global rule_defs
    results = []
    trials = []
    # Knowledge initialization
    init_participant_response_rules(participant)
    init_participant_construction_rules(participant)
    
    for _, trial in session_df.iterrows():
        trials.append(trial)
        break # testing
    
    participant.start_construct_trial(timedelta(seconds=1))
    while participant.system.queue:
        event = participant.system.advance()
        if event.source == participant.start_construct_trial:
            if not trials: break
            #load the next trial
            trial = trials.pop(0)
            grid_stimulus_np, grid_stimulus, test_query = load_trial(participant.construction_space, participant.response_space, trial, t_type=session_type, q_type=q_type)
            participant.construction_input.send(grid_stimulus) # TODO: have a timeout somehow: but how to do timeout wihout proper timinmg constraints for the various events in the queue?
        elif event.source == participant.end_construction:
            if session_type == "train":
                print("Construction was ", "correct" if np.all(grid_stimulus_np == numpify_grid(participant.construction_input.main[0])) else "incorrect")
                participant.propagate_feedback(correct = np.all(grid_stimulus_np == numpify_grid(participant.construction_input.main[0])))
            else:
                participant.start_response_trial(timedelta()) #TODO: checkout the actual time delays
        elif event.source == participant.end_construction_feedback:
            participant.start_response_trial(timedelta())
        elif event.source == participant.start_response_trial:
            participant.response_input.send(test_query)
        elif event.source == participant.response_rules.rules.rhs.td.update:
            participant.response_choice.select()
        elif event.source == participant.response_choice.select:
            results.append((event.time, participant.response_choice.poll())) #TODO: come up with a way to save the sequences of search space rules that were activated -- is there in sample attribute of the choice in a rule i believe?
            participant.finish_response_trial(timedelta())
        elif event.source == participant.finish_response_trial:
            participant.start_construct_trial(timedelta())

In [ ]:
trials_df = pd.read_csv("~/personalproj/clarion_replay/processed/test_data/all_test_data.csv")
run_participant_session(Participant("p1"), trials_df)

 15%|█▍        | 959363/6553600 [01:09<04:21, 21374.94it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1079070e0>>
Traceback (most recent call last):
  File "/Users/mishaal/miniconda3/envs/tapnseek/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 16%|█▋        | 1070221/6553600 [01:16<04:24, 20696.77it/s]

In [ ]:
np.where(np.array([[0, 0, 1], [0, 1, 1], [0, 0, 0], [0,0, 0], [0, 0, 0], [0, 0,0]]) == 1)

(array([0, 1, 1]), array([2, 1, 2]))